In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import re
from collections import defaultdict

import torchtext
from torchtext.legacy.data import Example

from wikipedia2vec import Wikipedia2Vec

In [4]:
# Data with Entities

ROOT_FOLDER = "/raid6/home/ramraj/2021/ir/entity-reranking/Entity-Linking/"

# BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "benchmark-train-relevance-v2.0")
# BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "test-data", "ramraj-test-data-top100-BM25.json")
# BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "Train-with-entities")
# BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "Test-with-entities/ramraj-test-data-top100-BM25-opt.json")

BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "Train-with-entities-sepEntAndWords")
BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "Test-with-entities/ramraj-test-data-top100-BM25-opt-entWordSEP.json")

# laod train

In [5]:
train_files = sorted(glob.glob(os.path.join(BENCHMARK_TRAIN_FOLD_FOLDER, "fold-*.json")))
print( len(train_files) )

train_data = []
for train_file in train_files:
    tmp_data = json.load(open(train_file, 'r'))
    train_data.extend(tmp_data)
    
len(train_data)

1


1937

# load test

In [6]:
test_data = json.load(open(BENCHMARK_TEST_FILE, 'r'))
len(test_data)

2254

# utils

In [7]:
# def tokenise(text):
#     # Replace annoying unicode with a space
#     text = re.sub(r'[^\x00-\x7F]+', ' ', text)
#     # The following replacements are suggested in the paper
#     # BidAF (Seo et al., 2016)
#     text = text.replace("''", '" ')
#     text = text.replace("``", '" ')

#     # Space out punctuation
#     space_list = "!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~"
#     space_list = "!\"#$%&()*+,./:;<=>?@[\\]^_`{|}~"
#     text = text.translate(str.maketrans({key: " {0} ".format(key) for key in space_list}))

#     # space out singlequotes a bit better (and like stanford)
#     text = text.replace("'", " '")
    
#     # use any APIs
#     text = text.replace('\t', '').replace('\n', '').lower().strip()
#     return text

In [8]:
# def alpha_numeric_filtering(text):
#     regex = re.compile('[^a-zA-Z]')
#     #First parameter is the replacement, second parameter is your input string
#     return regex.sub(' ', text)     

# load wikie

In [9]:
import torchtext.vocab as vocab

MODEL_FILE = "../entity-embedding-modelling/data/enwiki_20180420_300d.txt"
# https://github.com/pytorch/text/issues/722

custom_embeddings = vocab.Vectors(name = MODEL_FILE, cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")

In [10]:
# PKL_MODEL_FILE = "../entity-embedding-modelling/data/enwiki_20180420_100d.pkl"
# wiki2vec = Wikipedia2Vec.load(PKL_MODEL_FILE)

# vocab

In [11]:
PAD_TOKEN = '<PAD>'
SOS_TOKEN = '<SOS>'
UNK_TOKEN = '<UNK>'
EOS_TOKEN = '<EOS>'

MAX_Q_LENGTH = 20
MAX_D_LEN = 200

Q_TEXT = torchtext.legacy.data.Field(sequential=True, 
                                tokenize=lambda x: x.split('<SEP>'),
                                eos_token=EOS_TOKEN,
                                pad_token=PAD_TOKEN,
                                include_lengths=True,
                                batch_first=True,
                                fix_length=MAX_Q_LENGTH,
                                lower=True)

DOC_TEXT = torchtext.legacy.data.Field(sequential=True, 
                                tokenize=lambda x: x.split('<SEP>'),
                                eos_token=EOS_TOKEN,
                                pad_token=PAD_TOKEN,
                                include_lengths=True,
                                batch_first=True,
                                fix_length=MAX_D_LEN,
                                lower=True)

In [12]:
example_texts = []

FIELDS = [('doc_text', DOC_TEXT), ('q_text', Q_TEXT)]
# https://github.com/pytorch/text/issues/722


for train_data_sample in train_data:
    q_text = train_data_sample['qString']
    # q_text = '<SEP>'.join( [ e['entity_title'] for e in train_data_sample['qEntities'] ] )
    
    for rel_docs in train_data_sample['RelevantDocuments']:
        doc_text = rel_docs['docText']
        # dEntities = [ e['entity_title'] for e in rel_docs['dEntities'] ]
        # doc_text = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )

        
        example_texts.append( Example.fromlist([doc_text, q_text] , FIELDS ) )
        
for test_data_sample in test_data:
    q_text = test_data_sample['qString']
    # q_text = '<SEP>'.join( [ e['entity_title'] for e in test_data_sample['qEntities'] ] )
    
    for rel_docs in test_data_sample['RelevantDocuments']:
        doc_text = rel_docs['docText']
        # doc_text = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
        
        example_texts.append( Example.fromlist([doc_text, q_text] , FIELDS ) )

In [13]:
torchtext_dataset = torchtext.legacy.data.Dataset(example_texts, fields=FIELDS)

Q_TEXT.build_vocab(torchtext_dataset, vectors=custom_embeddings, vectors_cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")
DOC_TEXT.build_vocab(torchtext_dataset, vectors=custom_embeddings, vectors_cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")

print("======= Query ========")
print(len(Q_TEXT.vocab) )
print(Q_TEXT.vocab.vectors.size() )
print("======= Doc ========")
print( len(DOC_TEXT.vocab))
print( DOC_TEXT.vocab.vectors.size())

======= Query ========
4004
torch.Size([4004, 300])
======= Doc ========
229696
torch.Size([229696, 300])


In [14]:
print( DOC_TEXT.vocab.stoi['university of southampton'], DOC_TEXT.vocab.itos[14] )
print( DOC_TEXT.vocab.stoi['fahrenheit'], DOC_TEXT.vocab.itos[3] )

18345 new
16342 


# dataset

In [15]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

from pytorch_transformers import WarmupLinearSchedule, AdamW
from transformers import get_linear_schedule_with_warmup


def map_to_torch_float(encoding):
    encoding = torch.FloatTensor(encoding)
    encoding.requires_grad_(False)
    return encoding

def map_to_torch(encoding):
    encoding = torch.LongTensor(encoding)
    encoding.requires_grad_(False)
    return encoding


class TrainTRECDataset(Dataset):
    def __init__(self, raw_data, q_field, d_field, query_max_len=20, max_len=512, is_train=True):
        self.is_train = is_train
        self.max_len = max_len
        self.query_max_len = query_max_len
        
        self.q_field = q_field
        self.d_field = d_field

        self.qLookup = defaultdict()
        self.dLookup = defaultdict()
        self.data_df = self.load_data(raw_data)
        self.len = len(self.data_df)

    def load_data(self, data):
        return_data = {'qID': [], 'qText': [], 'dID': [], 'dText': [], 'label': []}
        for data_sample in data:
            qID = data_sample['qID']
            qText = data_sample['qString']
            # qText = '<SEP>'.join( [ e['entity_title'] for e in data_sample['qEntities'] ] )

            if qText == '': continue
            self.qLookup[qID] = qText

            for rel_docs in data_sample['RelevantDocuments']:
                dID = rel_docs['docID']
                docText = rel_docs['docText']
                # docText = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
                
                if docText == '': continue
                self.dLookup[dID] = docText

                return_data['qID'].append(qID)
                return_data['dID'].append(dID)
                # return_data['qText'].append( qText )
                # return_data['dText'].append( docText )

        # df = pd.DataFrame(return_data, columns=["qID", "qText", "dID", "dText"])
        df = pd.DataFrame(return_data, columns=["qID", "dID"])
        return df

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        index = index % self.len
        inst = self.data_df.loc[index]
        docID = inst['dID']
        qID = inst['qID']
        docText = self.dLookup[docID] # self.dLookup[docID] # inst['dText']
        qText = self.qLookup[qID] # self.qLookup[qID] # inst['qText']

        negInst = self.data_df[self.data_df['qID'] != qID].sample(n=3, replace=True, random_state=1).iloc[0]
        # negInst = self.data_df[self.data_df['qID'] != qID].iloc[0] # get a random element
        negDocID = negInst['dID']
        negDocText = self.dLookup[negDocID] # self.dLookup[negDocID] # negInst['dText']

        # =============== prepare text before feed ===============
        q_text_processed = self.q_field.numericalize( self.q_field.pad([ [l for l in self.q_field.preprocess(qText) if l.strip()] ]) )
        pos_d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(docText) if l.strip()] ]) )
        neg_d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(negDocText) if l.strip()] ]) )
        
        label = 1
        return q_text_processed, pos_d_text_processed, neg_d_text_processed, map_to_torch_float([label])


train_trec_dataset = TrainTRECDataset(train_data, Q_TEXT, DOC_TEXT)
print( train_trec_dataset.__len__() )

q, pos_d, neg_d, l = train_trec_dataset.__getitem__(1)

print(q[0].shape)
print(pos_d[0].shape)
print(neg_d[0].shape)
# print(q[0][0].shape)
# print(out[1][0].shape)
# print(out[2][0].shape)
# print()
# print(out[0][0].shape)

4863
torch.Size([1, 20])
torch.Size([1, 200])
torch.Size([1, 200])


In [16]:
q_str = []
for qq in q[0][0]:
    q_str.append(Q_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in pos_d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in neg_d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

fightorflight response urination evolutionary psychology <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

evolutionary psychology explanation early animals react threatening stimulus (physiology) quickly time psychological abuse physical abuse prepare fightorflight response provided mechanisms rapidly respond biodiversity life <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [17]:
import random
from bunch import Bunch
import time
import datetime



CONFIG_FILE = "config.json"

def get_config_from_json(json_file):
    """
        Get the config from a json file
        :param json_file:
        :return: config(namespace) or config(dictionary)
        """
    # parse the configurations from the config json file provided
    with open(json_file, 'r') as config_file:
        config_dict = json.load(config_file)

    # convert the dictionary to a namespace using bunch lib
    config = Bunch(config_dict)

    return config, config_dict

import torch
config, _ = get_config_from_json(CONFIG_FILE)



config.data['results_dir'] = 'exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly_moreEpochs'




seed_val = config.cmd_args['seed']
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
os.makedirs(config.data['results_dir'], exist_ok=True)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


train_dataloader = DataLoader(dataset=train_trec_dataset,
                              batch_size=config.training["train_batch_size"],
                              pin_memory=device,
                              num_workers=config.training['num_workers'],
                              shuffle=True)
n_train_batches = len(train_dataloader)
print("Number of train batches : ", n_train_batches)

cuda
Number of train batches :  304


In [19]:
for batch_data in train_dataloader:
    q, pos_d, neg_d, l = batch_data
    print(torch.squeeze(q[0]).shape)
    print(pos_d[0].shape)
    print(neg_d[0].shape)
    break

torch.Size([16, 20])
torch.Size([16, 1, 200])
torch.Size([16, 1, 200])


# Model

In [20]:
import typing
import torch.nn as nn
import torch.nn.functional as F


class SimScorer(nn.Module):
    def __init__(self, q_text_field, d_text_field, with_multi_layer_perceptron=True, mlp_num_layers=2, mlp_num_units=300, mlp_num_fan_out=1, padding_idx=1, embedding_freeze=True):
        super(SimScorer, self).__init__()
        self.with_multi_layer_perceptron = with_multi_layer_perceptron

        self.mlp_num_layers = mlp_num_layers
        self.mlp_num_units = mlp_num_units
        self.mlp_num_fan_out = mlp_num_fan_out
        self.q_text_field = q_text_field
        self.d_text_field = d_text_field
        self.padding_idx = padding_idx
        self.embedding_freeze = embedding_freeze
        self.build()
            
    def build(self):
        """
        Build model structure.
        """
        self.q_embedding = nn.Embedding.from_pretrained(self.q_text_field.vocab.vectors, padding_idx=self.padding_idx, freeze=self.embedding_freeze)
        self.d_embedding = nn.Embedding.from_pretrained(self.d_text_field.vocab.vectors, padding_idx=self.padding_idx, freeze=self.embedding_freeze)
        
        self.q_fc1 = self._make_perceptron_layer(300, 300)
        self.d_fc1 = self._make_perceptron_layer(300, 300)
        
    def _make_perceptron_layer(self, in_features=1, out_features=1, activation=nn.ReLU()):
        return nn.Sequential(nn.Linear(in_features, out_features), activation)

    def forward(self, inputs):
        input_left, input_right = inputs['text_left'], inputs['text_right']
        
        input_left_emb = self.q_embedding(input_left)
        input_right_emb = self.d_embedding(input_right)
        
#         print("input_left_emb : ", input_left_emb.shape)
#         print("input_right_emb : ", input_right_emb.shape)
        
        input_left_emb = self.q_fc1(input_left_emb)
        input_right_emb = self.d_fc1(input_right_emb)
        
#         print("input_left_emb : ", input_left_emb.shape)
#         print("input_right_emb : ", input_right_emb.shape)
        
        input_left_emb_avg = torch.mean(input_left_emb, dim=1)
        input_right_emb_avg = torch.mean(input_right_emb, dim=1)
        
        # print("input_left_emb_avg : ", input_left_emb_avg.shape)
        # print("input_right_emb_avg : ", input_right_emb_avg.shape)
    
        # Dot product with cosine similarity.
        x = F.cosine_similarity(input_left_emb_avg, input_right_emb_avg)
        # x = x.unsqueeze(dim=1)
        # print("x : ", x.shape)

        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

net = SimScorer(Q_TEXT, DOC_TEXT)
net.to(device)
print('Models built and ready to go!')

# tmp_q_text = torch.randint(low=1, high=30, size=(16, 20)).to(device)
# tmp_d_text = torch.randint(low=1, high=30, size=(16, 200)).to(device)
# print(tmp_q_text.shape)
# print(tmp_d_text.shape)

# input_text = {'text_left': tmp_q_text, 'text_right': tmp_d_text}

# output = net.forward(input_text)
# print(output)
# output.shape

cuda
Models built and ready to go!


In [21]:
LEARNING_RATE = 0.001 # 0.001 # 2e-5

criterion = nn.MarginRankingLoss(margin=1, size_average=True)
# criterion = nn.MarginRankingLoss(size_average=True)
opti = AdamW(net.parameters(),
             lr=LEARNING_RATE,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
             eps=1e-8,  # args.adam_epsilon  - default is 1e-8.
             correct_bias=False
             )

num_epochs = config.training['epochs']
display_step = config['training']['display_step']
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(opti,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps)

def format_time(elapsed_time):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed_time)))
    return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss

/raid6/home/ramraj/anaconda3/envs/ir-research-py3.8/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [22]:
training_stats = []
history = defaultdict(list)

resume_epoch = 0
print_every = 1 # 10
save_every = 200 # 1000

print('Initializing ...')
print("Training...")


# resume_epoch = 11
# num_epochs = 20

total_t0 = time.time()
for epoch_idx in range(resume_epoch, num_epochs):
    total_loss = 0
    total_score = 0


    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_idx + 1, num_epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    net.train()  # TODO: IMPORTANT !
    for batch_idx, train_batch_data in enumerate(train_dataloader):
        # Clear gradients
        net.zero_grad()  # TODO: check validity !
        opti.zero_grad()
        
        # Converting these to cuda tensors
        q_text, pos_d_text, neg_d_text, label = train_batch_data

        q_text, pos_d_text, neg_d_text, label = torch.squeeze(q_text[0]).to(device), torch.squeeze(pos_d_text[0]).to(device), torch.squeeze(neg_d_text[0]).to(device), label.to(device)

        pos_net_output = net({'text_left': q_text, 'text_right': pos_d_text})
        neg_net_output = net({'text_left': q_text, 'text_right': neg_d_text})
        # # TODO: do i need a softmax or not ?

        # Computing loss
        loss = criterion(pos_net_output, neg_net_output, label.float())
        batch_loss = loss.item()
        total_loss += batch_loss

        # Back propagating the gradients
        loss.backward()
        if config.training['gradient_clipping']['use']:
            torch.nn.utils.clip_grad_norm_(net.parameters(), config.training['gradient_clipping']['clip_value'])

        # Optimization step
        opti.step()        

        with open(os.path.join(config.data['results_dir'], 'train-log-epoch.txt' ), 'a') as f:
                f.write(str(batch_idx+1) + '\t' + str(batch_idx+1) + '\t' + str(batch_loss) + '\t' + '\n')

        training_stats.append(
                {
                    'epoch': epoch_idx + 1,
                    'batch': batch_idx + 1,
                    'step': (epoch_idx * n_train_batches) + batch_idx + 1,
                    'Training Loss': batch_loss,
                    # 'Training Loss': avg_train_loss,
                    # 'Training Time': training_time,
                }
            )
                
        if batch_idx % print_every == 0: # Print progress
            total_loss_avg = total_loss / print_every 
            elapsed = format_time(time.time() - t0)
            print('| TRAIN SET | Epoch [{:02d}/{:02d}], Step [{:04d}/{:04d}], Loss: {:.4f} | Elapsed: {:}'
                          .format(epoch_idx+1, num_epochs, batch_idx, int(n_train_batches), total_loss_avg, elapsed))
            total_loss = 0              
        
        if ( (batch_idx == n_train_batches-1) or ((batch_idx+1) % save_every == 0) ): # Save checkpoint
            directory = os.path.join(config.data['results_dir'], 'ranking-pytorch-model')
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save(net.state_dict(),
                       os.path.join(directory, 'epoch-{}.batch-{}.{}.pt'.format(epoch_idx+1, batch_idx+1, 'checkpoint')))


    scheduler.step()  # TODO: IMPORTANT !

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

history_df = pd.DataFrame({"step": [e['step'] for e in training_stats],
                           "Training Loss": [e['Training Loss'] for e in training_stats]})
history_df.to_csv(os.path.join(config.data['results_dir'], "history.csv"), index=False)

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))

if not config.cmd_args['mode'] == "experiment":
    torch.save(net, os.path.join(config.data['results_dir'], "model-dump.pkl"))

Initializing ...
Training...

======== Epoch 1 / 4 ========
Training...


/raid6/home/ramraj/anaconda3/envs/ir-research-py3.8/lib/python3.8/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


| TRAIN SET | Epoch [01/04], Step [0000/0304], Loss: 0.9747 | Elapsed: 0:00:00
| TRAIN SET | Epoch [01/04], Step [0001/0304], Loss: 0.8566 | Elapsed: 0:00:00
| TRAIN SET | Epoch [01/04], Step [0002/0304], Loss: 0.7943 | Elapsed: 0:00:00
| TRAIN SET | Epoch [01/04], Step [0003/0304], Loss: 0.6034 | Elapsed: 0:00:00
| TRAIN SET | Epoch [01/04], Step [0004/0304], Loss: 0.5351 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0005/0304], Loss: 0.3417 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0006/0304], Loss: 0.4072 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0007/0304], Loss: 0.2064 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0008/0304], Loss: 0.2480 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0009/0304], Loss: 0.2852 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0010/0304], Loss: 0.1885 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0011/0304], Loss: 0.0832 | Elapsed: 0:00:01
| TRAIN SET | Epoch [01/04], Step [0012/0304], Loss:

| TRAIN SET | Epoch [01/04], Step [0106/0304], Loss: 0.1894 | Elapsed: 0:00:05
| TRAIN SET | Epoch [01/04], Step [0107/0304], Loss: 0.0642 | Elapsed: 0:00:05
| TRAIN SET | Epoch [01/04], Step [0108/0304], Loss: 0.0018 | Elapsed: 0:00:05
| TRAIN SET | Epoch [01/04], Step [0109/0304], Loss: 0.0665 | Elapsed: 0:00:05
| TRAIN SET | Epoch [01/04], Step [0110/0304], Loss: 0.0022 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0111/0304], Loss: 0.0644 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0112/0304], Loss: 0.0019 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0113/0304], Loss: 0.0638 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0114/0304], Loss: 0.1268 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0115/0304], Loss: 0.1269 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0116/0304], Loss: 0.0636 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0117/0304], Loss: 0.0021 | Elapsed: 0:00:06
| TRAIN SET | Epoch [01/04], Step [0118/0304], Loss:

| TRAIN SET | Epoch [01/04], Step [0212/0304], Loss: 0.0012 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0213/0304], Loss: 0.0015 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0214/0304], Loss: 0.0637 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0215/0304], Loss: 0.0011 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0216/0304], Loss: 0.0635 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0217/0304], Loss: 0.0013 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0218/0304], Loss: 0.0017 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0219/0304], Loss: 0.1254 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0220/0304], Loss: 0.0631 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0221/0304], Loss: 0.0012 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0222/0304], Loss: 0.0012 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0223/0304], Loss: 0.0637 | Elapsed: 0:00:12
| TRAIN SET | Epoch [01/04], Step [0224/0304], Loss:

...

| TRAIN SET | Epoch [03/04], Step [0132/0304], Loss: 0.0630 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0133/0304], Loss: 0.0007 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0134/0304], Loss: 0.0624 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0135/0304], Loss: 0.0010 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0136/0304], Loss: 0.0014 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0137/0304], Loss: 0.0612 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0138/0304], Loss: 0.0629 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0139/0304], Loss: 0.0008 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0140/0304], Loss: 0.0009 | Elapsed: 0:00:07
| TRAIN SET | Epoch [03/04], Step [0141/0304], Loss: 0.0632 | Elapsed: 0:00:08
| TRAIN SET | Epoch [03/04], Step [0142/0304], Loss: 0.0008 | Elapsed: 0:00:08
| TRAIN SET | Epoch [03/04], Step [0143/0304], Loss: 0.0631 | Elapsed: 0:00:08
| TRAIN SET | Epoch [03/04], Step [0144/0304], Loss:

| TRAIN SET | Epoch [03/04], Step [0236/0304], Loss: 0.0009 | Elapsed: 0:00:13
| TRAIN SET | Epoch [03/04], Step [0237/0304], Loss: 0.0627 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0238/0304], Loss: 0.0634 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0239/0304], Loss: 0.1258 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0240/0304], Loss: 0.1254 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0241/0304], Loss: 0.0008 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0242/0304], Loss: 0.0638 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0243/0304], Loss: 0.0006 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0244/0304], Loss: 0.0638 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0245/0304], Loss: 0.0012 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0246/0304], Loss: 0.0014 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0247/0304], Loss: 0.0623 | Elapsed: 0:00:14
| TRAIN SET | Epoch [03/04], Step [0248/0304], Loss:

| TRAIN SET | Epoch [04/04], Step [0038/0304], Loss: 0.0008 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0039/0304], Loss: 0.0007 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0040/0304], Loss: 0.1257 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0041/0304], Loss: 0.1236 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0042/0304], Loss: 0.0009 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0043/0304], Loss: 0.1241 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0044/0304], Loss: 0.0006 | Elapsed: 0:00:02
| TRAIN SET | Epoch [04/04], Step [0045/0304], Loss: 0.0011 | Elapsed: 0:00:03
| TRAIN SET | Epoch [04/04], Step [0046/0304], Loss: 0.1250 | Elapsed: 0:00:03
| TRAIN SET | Epoch [04/04], Step [0047/0304], Loss: 0.0634 | Elapsed: 0:00:03
| TRAIN SET | Epoch [04/04], Step [0048/0304], Loss: 0.0008 | Elapsed: 0:00:03
| TRAIN SET | Epoch [04/04], Step [0049/0304], Loss: 0.0638 | Elapsed: 0:00:03
| TRAIN SET | Epoch [04/04], Step [0050/0304], Loss:

| TRAIN SET | Epoch [04/04], Step [0144/0304], Loss: 0.1252 | Elapsed: 0:00:07
| TRAIN SET | Epoch [04/04], Step [0145/0304], Loss: 0.0008 | Elapsed: 0:00:07
| TRAIN SET | Epoch [04/04], Step [0146/0304], Loss: 0.0013 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0147/0304], Loss: 0.0615 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0148/0304], Loss: 0.0010 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0149/0304], Loss: 0.2447 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0150/0304], Loss: 0.0012 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0151/0304], Loss: 0.1131 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0152/0304], Loss: 0.0559 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0153/0304], Loss: 0.0637 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0154/0304], Loss: 0.1765 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0155/0304], Loss: 0.0492 | Elapsed: 0:00:08
| TRAIN SET | Epoch [04/04], Step [0156/0304], Loss:

| TRAIN SET | Epoch [04/04], Step [0250/0304], Loss: 0.0638 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0251/0304], Loss: 0.0627 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0252/0304], Loss: 0.0601 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0253/0304], Loss: 0.1685 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0254/0304], Loss: 0.2017 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0255/0304], Loss: 0.1898 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0256/0304], Loss: 0.0024 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0257/0304], Loss: 0.0034 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0258/0304], Loss: 0.0628 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0259/0304], Loss: 0.0027 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0260/0304], Loss: 0.0639 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0261/0304], Loss: 0.1269 | Elapsed: 0:00:14
| TRAIN SET | Epoch [04/04], Step [0262/0304], Loss:

# Test Data

In [23]:
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# import torch

# from pytorch_transformers import WarmupLinearSchedule, AdamW
# from transformers import get_linear_schedule_with_warmup

# def map_to_torch_float(encoding):
#     encoding = torch.FloatTensor(encoding)
#     encoding.requires_grad_(False)
#     return encoding

# def map_to_torch(encoding):
#     encoding = torch.LongTensor(encoding)
#     encoding.requires_grad_(False)
#     return encoding


class TestTRECDataset(Dataset):
    def __init__(self, raw_data, q_field, d_field, query_max_len=20, max_len=512, is_train=True):
        self.is_train = is_train
        self.max_len = max_len
        self.query_max_len = query_max_len
        
        self.q_field = q_field
        self.d_field = d_field

        self.qLookup = defaultdict()
        self.dLookup = defaultdict()
        self.data_df = self.load_data(raw_data)
        self.len = len(self.data_df)
        
    def load_data(self, data):
        return_data = {'qID': [], 'qText': [], 'dID': [], 'dText': [], 'label': []}
        for data_sample in data:
            qID = data_sample['qID']
            qText = data_sample['qString']
            #qText = '<SEP>'.join( [ e['entity_title'] for e in data_sample['qEntities'] ] )

            if qText == '': continue
            self.qLookup[qID] = qText

            for rel_docs in data_sample['RelevantDocuments']:
                dID = rel_docs['docID']
                docText = rel_docs['docText']
                #docText = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
                
                if docText == '': continue
                self.dLookup[dID] = docText

                return_data['qID'].append(qID)
                return_data['dID'].append(dID)
                # return_data['qText'].append( qText )
                # return_data['dText'].append( docText )

        # df = pd.DataFrame(return_data, columns=["qID", "qText", "dID", "dText"])
        df = pd.DataFrame(return_data, columns=["qID", "dID"])
        return df
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        index = index % self.len
        inst = self.data_df.loc[index]
        docID = inst['dID']
        qID = inst['qID']
        docText = self.dLookup[docID] # self.dLookup[docID] # inst['dText']
        qText = self.qLookup[qID] # self.qLookup[qID] # inst['qText']
        
        # print(qID)
        # print(qText)
        # print()
        # print(docID)
        # print(docText)

        # =============== prepare text before feed ===============
        q_text_processed = self.q_field.numericalize( self.q_field.pad([ [l for l in self.q_field.preprocess(qText) if l.strip()] ]) )
        d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(docText) if l.strip()] ]) )

        return q_text_processed, d_text_processed, qID, docID
    

test_trec_dataset = TestTRECDataset(test_data, Q_TEXT, DOC_TEXT)
print( test_trec_dataset.__len__() )

q, d, qID, dID = test_trec_dataset.__getitem__(1)

print(q[0].shape)
print(d[0].shape)
# print(q[0][0].shape)
# print(out[1][0].shape)
# print(out[2][0].shape)
# print()
# print(out[0][0].shape)

225156
torch.Size([1, 20])
torch.Size([1, 200])


In [24]:
q_str = []
for qq in q[0][0]:
    q_str.append(Q_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

aftertaste <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

aftertaste taste intensity food drink perception immediately food drink removed mouth aftertaste different food drink tin can vary intensity time unifying feature aftertaste perception food drink either swallowed spat neuroscience evolution taste aftertaste signal transduction taste receptor mouth brain yet fully understood however primary taste food processing area located insular cortex observed involved aftertaste perception <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

In [25]:
test_dataloader = DataLoader(dataset=test_trec_dataset,
                              batch_size=config.training["test_batch_size"],
                              pin_memory=device,
                              num_workers=config.training['num_workers'],
                              shuffle=True)
n_test_batches = len(test_dataloader)
print("Number of test batches : ", n_test_batches)
net.eval()

Number of test batches :  880


SimScorer(
  (q_embedding): Embedding(4004, 300, padding_idx=1)
  (d_embedding): Embedding(229696, 300, padding_idx=1)
  (q_fc1): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
  )
  (d_fc1): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
  )
)

# test

In [26]:
qID_list = []
paraID_list = []
pScore_list = []
t1 = time.time()
for batch_idx, test_batch_data in enumerate(test_dataloader):
    # Converting these to cuda tensors
    q_text, d_text, qID, passageID = test_batch_data
#     print(type(q_text))
#     x = q_text[0]
#     y = q_text[1]
#     print(x.shape)
#     print(y.shape)
#     break
    q_text, d_text = torch.squeeze(q_text[0].to(device)), torch.squeeze(d_text[0]).to(device)

    with torch.no_grad():
        net_output = net({'text_left': q_text, 'text_right': d_text})
        net_output = net_output.detach().cpu().numpy()

        for i in range(len(qID)):
            qID_list.append(qID[i])
            paraID_list.append(passageID[i])
            pScore_list.append(net_output[i])
    elapsed = format_time(time.time() - t1)
    if batch_idx % 50 == 0:
        print('  Batch {:>5,}  of  {:>5,}  :  processed    Elapsed: {:}.'.format(batch_idx,
                                                                                 n_test_batches,
                                                                                 elapsed))

pScore_list = [float(e) for e in pScore_list]
predicted_df = pd.DataFrame({"qID": qID_list,
                             "pID": paraID_list,
                             "pScore": pScore_list}, columns=["qID", "pID", "pScore"])
if not config.cmd_args['mode'] == "experiment":
    predicted_df.to_csv(os.path.join(config.data['results_dir'], "predictions.csv"))
print()

  Batch     0  of    880  :  processed    Elapsed: 0:00:01.
  Batch    50  of    880  :  processed    Elapsed: 0:00:06.
  Batch   100  of    880  :  processed    Elapsed: 0:00:11.
  Batch   150  of    880  :  processed    Elapsed: 0:00:16.
  Batch   200  of    880  :  processed    Elapsed: 0:00:22.
  Batch   250  of    880  :  processed    Elapsed: 0:00:28.
  Batch   300  of    880  :  processed    Elapsed: 0:00:34.
  Batch   350  of    880  :  processed    Elapsed: 0:00:40.
  Batch   400  of    880  :  processed    Elapsed: 0:00:46.
  Batch   450  of    880  :  processed    Elapsed: 0:00:52.
  Batch   500  of    880  :  processed    Elapsed: 0:01:01.
  Batch   550  of    880  :  processed    Elapsed: 0:01:06.
  Batch   600  of    880  :  processed    Elapsed: 0:01:12.
  Batch   650  of    880  :  processed    Elapsed: 0:01:18.
  Batch   700  of    880  :  processed    Elapsed: 0:01:24.
  Batch   750  of    880  :  processed    Elapsed: 0:01:30.
  Batch   800  of    880  :  processed  

In [27]:
# ================================================
#               Reverse Sorting Relevance
# ================================================
predicted_df = predicted_df[['qID', 'pID', 'pScore']]
grouped_pred_df = predicted_df.groupby(["qID"])
num_queries = len(grouped_pred_df)
missing_q_sets = 0
save_ranked_file = os.path.join(config.data['results_dir'], "ranked.test.relevance.txt")
with open(save_ranked_file, 'w') as write_file:
    q_cnt = 1
    for name, row_group in grouped_pred_df:
        rank_cnt = 1

        # ======= SORTING =======
        sorted_row_group = row_group.sort_values(by='pScore', ascending=False, inplace=False)
        # =======================

        if len(sorted_row_group) != 100:
            # print(">>>>>>>>>>> Missing query %s with shape %s" % (name, sorted_row_group.shape))
            # print(">>>>>>>>>>> Missing query with size %s" % sorted_row_group.shape[0])
            missing_q_sets += 1

        for i, row in sorted_row_group.iterrows():
            write_file.write("%s\tQ0\t%s\t%s\t%s\trchan31\n" % \
                             (row["qID"], row["pID"], rank_cnt, row["pScore"]))
            rank_cnt += 1

        print("Finished composing for query number : %s / %s" % (q_cnt, num_queries))
        q_cnt += 1
print()
print("Missing query-doc pairs : ", missing_q_sets)
print("Done train, val, and test !!!")

Finished composing for query number : 1 / 2254
Finished composing for query number : 2 / 2254
Finished composing for query number : 3 / 2254
Finished composing for query number : 4 / 2254
Finished composing for query number : 5 / 2254
Finished composing for query number : 6 / 2254
Finished composing for query number : 7 / 2254
Finished composing for query number : 8 / 2254
Finished composing for query number : 9 / 2254
Finished composing for query number : 10 / 2254
Finished composing for query number : 11 / 2254
Finished composing for query number : 12 / 2254
Finished composing for query number : 13 / 2254
Finished composing for query number : 14 / 2254
Finished composing for query number : 15 / 2254
Finished composing for query number : 16 / 2254
Finished composing for query number : 17 / 2254
Finished composing for query number : 18 / 2254
Finished composing for query number : 19 / 2254
Finished composing for query number : 20 / 2254
Finished composing for query number : 21 / 2254
F

Finished composing for query number : 172 / 2254
Finished composing for query number : 173 / 2254
Finished composing for query number : 174 / 2254
Finished composing for query number : 175 / 2254
Finished composing for query number : 176 / 2254
Finished composing for query number : 177 / 2254
Finished composing for query number : 178 / 2254
Finished composing for query number : 179 / 2254
Finished composing for query number : 180 / 2254
Finished composing for query number : 181 / 2254
Finished composing for query number : 182 / 2254
Finished composing for query number : 183 / 2254
Finished composing for query number : 184 / 2254
Finished composing for query number : 185 / 2254
Finished composing for query number : 186 / 2254
Finished composing for query number : 187 / 2254
Finished composing for query number : 188 / 2254
Finished composing for query number : 189 / 2254
Finished composing for query number : 190 / 2254
Finished composing for query number : 191 / 2254
Finished composing f

Finished composing for query number : 351 / 2254
Finished composing for query number : 352 / 2254
Finished composing for query number : 353 / 2254
Finished composing for query number : 354 / 2254
Finished composing for query number : 355 / 2254
Finished composing for query number : 356 / 2254
Finished composing for query number : 357 / 2254
Finished composing for query number : 358 / 2254
Finished composing for query number : 359 / 2254
Finished composing for query number : 360 / 2254
Finished composing for query number : 361 / 2254
Finished composing for query number : 362 / 2254
Finished composing for query number : 363 / 2254
Finished composing for query number : 364 / 2254
Finished composing for query number : 365 / 2254
Finished composing for query number : 366 / 2254
Finished composing for query number : 367 / 2254
Finished composing for query number : 368 / 2254
Finished composing for query number : 369 / 2254
Finished composing for query number : 370 / 2254
Finished composing f

Finished composing for query number : 527 / 2254
Finished composing for query number : 528 / 2254
Finished composing for query number : 529 / 2254
Finished composing for query number : 530 / 2254
Finished composing for query number : 531 / 2254
Finished composing for query number : 532 / 2254
Finished composing for query number : 533 / 2254
Finished composing for query number : 534 / 2254
Finished composing for query number : 535 / 2254
Finished composing for query number : 536 / 2254
Finished composing for query number : 537 / 2254
Finished composing for query number : 538 / 2254
Finished composing for query number : 539 / 2254
Finished composing for query number : 540 / 2254
Finished composing for query number : 541 / 2254
Finished composing for query number : 542 / 2254
Finished composing for query number : 543 / 2254
Finished composing for query number : 544 / 2254
Finished composing for query number : 545 / 2254
Finished composing for query number : 546 / 2254
Finished composing f

Finished composing for query number : 710 / 2254
Finished composing for query number : 711 / 2254
Finished composing for query number : 712 / 2254
Finished composing for query number : 713 / 2254
Finished composing for query number : 714 / 2254
Finished composing for query number : 715 / 2254
Finished composing for query number : 716 / 2254
Finished composing for query number : 717 / 2254
Finished composing for query number : 718 / 2254
Finished composing for query number : 719 / 2254
Finished composing for query number : 720 / 2254
Finished composing for query number : 721 / 2254
Finished composing for query number : 722 / 2254
Finished composing for query number : 723 / 2254
Finished composing for query number : 724 / 2254
Finished composing for query number : 725 / 2254
Finished composing for query number : 726 / 2254
Finished composing for query number : 727 / 2254
Finished composing for query number : 728 / 2254
Finished composing for query number : 729 / 2254
Finished composing f

...

Finished composing for query number : 1414 / 2254
Finished composing for query number : 1415 / 2254
Finished composing for query number : 1416 / 2254
Finished composing for query number : 1417 / 2254
Finished composing for query number : 1418 / 2254
Finished composing for query number : 1419 / 2254
Finished composing for query number : 1420 / 2254
Finished composing for query number : 1421 / 2254
Finished composing for query number : 1422 / 2254
Finished composing for query number : 1423 / 2254
Finished composing for query number : 1424 / 2254
Finished composing for query number : 1425 / 2254
Finished composing for query number : 1426 / 2254
Finished composing for query number : 1427 / 2254
Finished composing for query number : 1428 / 2254
Finished composing for query number : 1429 / 2254
Finished composing for query number : 1430 / 2254
Finished composing for query number : 1431 / 2254
Finished composing for query number : 1432 / 2254
Finished composing for query number : 1433 / 2254


Finished composing for query number : 1587 / 2254
Finished composing for query number : 1588 / 2254
Finished composing for query number : 1589 / 2254
Finished composing for query number : 1590 / 2254
Finished composing for query number : 1591 / 2254
Finished composing for query number : 1592 / 2254
Finished composing for query number : 1593 / 2254
Finished composing for query number : 1594 / 2254
Finished composing for query number : 1595 / 2254
Finished composing for query number : 1596 / 2254
Finished composing for query number : 1597 / 2254
Finished composing for query number : 1598 / 2254
Finished composing for query number : 1599 / 2254
Finished composing for query number : 1600 / 2254
Finished composing for query number : 1601 / 2254
Finished composing for query number : 1602 / 2254
Finished composing for query number : 1603 / 2254
Finished composing for query number : 1604 / 2254
Finished composing for query number : 1605 / 2254
Finished composing for query number : 1606 / 2254


Finished composing for query number : 1757 / 2254
Finished composing for query number : 1758 / 2254
Finished composing for query number : 1759 / 2254
Finished composing for query number : 1760 / 2254
Finished composing for query number : 1761 / 2254
Finished composing for query number : 1762 / 2254
Finished composing for query number : 1763 / 2254
Finished composing for query number : 1764 / 2254
Finished composing for query number : 1765 / 2254
Finished composing for query number : 1766 / 2254
Finished composing for query number : 1767 / 2254
Finished composing for query number : 1768 / 2254
Finished composing for query number : 1769 / 2254
Finished composing for query number : 1770 / 2254
Finished composing for query number : 1771 / 2254
Finished composing for query number : 1772 / 2254
Finished composing for query number : 1773 / 2254
Finished composing for query number : 1774 / 2254
Finished composing for query number : 1775 / 2254
Finished composing for query number : 1776 / 2254


Finished composing for query number : 1936 / 2254
Finished composing for query number : 1937 / 2254
Finished composing for query number : 1938 / 2254
Finished composing for query number : 1939 / 2254
Finished composing for query number : 1940 / 2254
Finished composing for query number : 1941 / 2254
Finished composing for query number : 1942 / 2254
Finished composing for query number : 1943 / 2254
Finished composing for query number : 1944 / 2254
Finished composing for query number : 1945 / 2254
Finished composing for query number : 1946 / 2254
Finished composing for query number : 1947 / 2254
Finished composing for query number : 1948 / 2254
Finished composing for query number : 1949 / 2254
Finished composing for query number : 1950 / 2254
Finished composing for query number : 1951 / 2254
Finished composing for query number : 1952 / 2254
Finished composing for query number : 1953 / 2254
Finished composing for query number : 1954 / 2254
Finished composing for query number : 1955 / 2254


Finished composing for query number : 2100 / 2254
Finished composing for query number : 2101 / 2254
Finished composing for query number : 2102 / 2254
Finished composing for query number : 2103 / 2254
Finished composing for query number : 2104 / 2254
Finished composing for query number : 2105 / 2254
Finished composing for query number : 2106 / 2254
Finished composing for query number : 2107 / 2254
Finished composing for query number : 2108 / 2254
Finished composing for query number : 2109 / 2254
Finished composing for query number : 2110 / 2254
Finished composing for query number : 2111 / 2254
Finished composing for query number : 2112 / 2254
Finished composing for query number : 2113 / 2254
Finished composing for query number : 2114 / 2254
Finished composing for query number : 2115 / 2254
Finished composing for query number : 2116 / 2254
Finished composing for query number : 2117 / 2254
Finished composing for query number : 2118 / 2254
Finished composing for query number : 2119 / 2254


In [28]:
! ls 

averageOf_cosineSim_cleanedText.ipynb
averageOf_cosineSim_entityOnly.ipynb
config.json
exp1_sim
exp1_sim_trec_results_withTestVocab.txt
exp2_averageOf_cosineSim_entityOnly
exp2_averageOf_cosineSim_entityOnly_trec_results.txt
exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly
exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly_moreEpochs
exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly-trec_results.txt
extended1FCLayerTrain_averageOf_cosineSim_entityOnly-Copy1.ipynb
extended1FCLayerTrain_averageOf_cosineSim_entityOnly.ipynb


In [29]:
! ls exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly_moreEpochs

history.csv		   ranking-pytorch-model
ranked.test.relevance.txt  train-log-epoch.txt


In [32]:
! ../Eval/trec_eval-master/trec_eval ../Eval/qrelsY1-test.V2.0/automatic-test.pages.cbor-hierarchical.qrels exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly_moreEpochs/ranked.test.relevance.txt  
# -m all_trec



runid                 	all	rchan31
num_q                 	all	2254
num_ret               	all	225156
num_rel               	all	6192
num_rel_ret           	all	2375
map                   	all	0.0277
gm_map                	all	0.0017
Rprec                 	all	0.0124
bpref                 	all	0.4689
recip_rank            	all	0.0483
iprec_at_recall_0.00  	all	0.0509
iprec_at_recall_0.10  	all	0.0501
iprec_at_recall_0.20  	all	0.0472
iprec_at_recall_0.30  	all	0.0365
iprec_at_recall_0.40  	all	0.0297
iprec_at_recall_0.50  	all	0.0284
iprec_at_recall_0.60  	all	0.0201
iprec_at_recall_0.70  	all	0.0192
iprec_at_recall_0.80  	all	0.0163
iprec_at_recall_0.90  	all	0.0158
iprec_at_recall_1.00  	all	0.0158
P_5                   	all	0.0113
P_10                  	all	0.0109
P_15                  	all	0.0108
P_20                  	all	0.0105
P_30                  	all	0.0108
P_100                 	all	0.0105
P_200                 	all	0.0053
P_500                 	all	0.0021
P_1000             

In [33]:
! ../Eval/trec_eval-master/trec_eval ../Eval/qrelsY1-test.V2.0/automatic-test.pages.cbor-hierarchical.qrels exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly_moreEpochs/ranked.test.relevance.txt  -m all_trec > exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly_moreEpochs-trec_results_moreEpochs.txt


